In [1]:
import pandas as pd
import plotly.graph_objs as go
import os

In [2]:
PATH = 'data/nba_ftr_diffs.csv'
FTA_ANIMATION_PATH = 'plots/fta_animation/'
df = pd.read_csv(PATH, index_col=0)

In [3]:
#df = df.iloc[:, 1:]
df.head()

,PLAYER_NAME,CURR_GP,CURR_MIN,CURR_FTR,TEAM_ABBREVIATION,PAST_GP,PAST_MIN,PAST_FTR,FTR_DIFF,CURR_FTA,PAST_FTA,FTA_DIFF
0,Vlatko Cancar,3,23.0,0.500,DEN,9,21.0,0.000,0.500,6.3,0.0,6.3
1,Sekou Doumbouya,2,16.0,0.571,DET,21,269.0,0.163,0.408,8.8,2.8,6.0
2,Zeke Nnaji,9,109.0,0.333,DEN,10,23.0,0.000,0.333,4.0,0.0,4.0
3,Tacko Fall,7,19.0,0.500,BOS,4,42.0,0.211,0.289,7.4,3.5,3.9
4,KZ Okpala,10,63.0,0.348,MIA,13,166.0,0.081,0.267,4.6,1.3,3.3


In [4]:
TOP15_DF = df[
               (df['CURR_GP'] >= 10) &
               (df['CURR_MIN'] >= 240) &
               (df['PAST_MIN'] >= 240) &
               (df['PAST_GP'] >= 10)
           ].sort_values(by='PAST_FTR', ascending=False).iloc[:15]

# sort back to highest FTR first
TOP15_DF.sort_values(by='PAST_FTR', ascending=False, inplace=True)
TOP15_DF

,PLAYER_NAME,CURR_GP,CURR_MIN,CURR_FTR,TEAM_ABBREVIATION,PAST_GP,PAST_MIN,PAST_FTR,FTR_DIFF,CURR_FTA,PAST_FTA,FTA_DIFF
89,Giannis Antetokounmpo,21,686.0,0.667,MIL,21,690.0,0.610,0.057,11.2,10.7,0.5
94,Joel Embiid,13,422.0,0.636,PHI,18,574.0,0.585,0.051,10.7,12.6,-1.9
325,Trae Young,23,783.0,0.407,ATL,20,688.0,0.505,-0.098,6.1,10.8,-4.7
150,Jimmy Butler,17,582.0,0.497,MIA,10,307.0,0.481,0.016,9.1,8.7,0.4
351,Damian Lillard,20,714.0,0.280,POR,20,721.0,0.456,-0.176,4.5,7.7,-3.2
300,Luka Doncic,17,590.0,0.378,DAL,22,765.0,0.456,-0.078,5.7,8.0,-2.3
99,DeMar DeRozan,23,808.0,0.492,SAS,20,673.0,0.441,0.051,7.8,7.2,0.6
161,Bam Adebayo,18,592.0,0.448,MIA,20,659.0,0.440,0.008,6.8,6.6,0.2
295,Shai Gilgeous-Alexander,19,663.0,0.366,OKC,19,633.0,0.438,-0.072,5.8,6.8,-1.0
354,Andre Drummond,22,471.0,0.251,LAL,21,622.0,0.434,-0.183,3.6,6.7,-3.1


In [5]:
N_FRAMES = 10
FIRST_FRAMES = N_FRAMES * 0.3
LAST_FRAMES = N_FRAMES - FIRST_FRAMES
N_TRANSITION_FRAMES = N_FRAMES - 2 * (FIRST_FRAMES)

In [75]:
# Create opacities for each frame
OPACITIES = list()

# Create dot opacities for first plot
for i in range(1, N_FRAMES+1):
    if i <= FIRST_FRAMES:
        OPACITIES.append(1)
    elif (i > FIRST_FRAMES) & (i <= LAST_FRAMES):
        transition_n = i - FIRST_FRAMES
        print(transition_n)
        frame_opacity = 1 - (transition_n/(N_TRANSITION_FRAMES+1))
        print(frame_opacity)
        OPACITIES.append(frame_opacity)
    else:
        OPACITIES.append(0)

1.0
0.8
2.0
0.6
3.0
0.4
4.0
0.19999999999999996


In [73]:
OPACITIES

[1, 1, 1, 0.8, 0.6, 0.4, 0.19999999999999996, 0, 0, 0]

In [33]:
# Draw one example figure indexing opacities
# fig = go.Figure()
#
# fig.add_trace(
#     go.Scatter(
#         x = TOP15_DF['PAST_FTR'],
#         y = TOP15_DF['PLAYER_NAME'],
#         mode='markers',
#         marker=dict(
#             opacity=0.5,
#             size=18,
#             color='#d4d4d4'
#         )
#     )
# )
#
# fig.add_trace(
#     go.Scatter(
#         x = TOP15_DF['CURR_FTR'],
#         y = TOP15_DF['PLAYER_NAME'],
#         mode='markers',
#         marker = dict(
#             opacity = 0.8,
#             size=18
# )
#     )
# )
#
# fig.update_layout(
#     # dimensions
#     height = 800,
#     width = 1000,
#     margin = dict(
#         t = 50,
#         b = 50,
#         r = 10,
#         l = 50,
#         pad = 0
#     ),
#
#     # axes info
#     xaxis = dict(
#         domain = [0.2, 0.8],
#         showgrid = False,
#     ),
#     yaxis = dict(
#         categoryorder = 'total ascending',
#         anchor = 'free',
#         position = 0.05,
#         side = 'right',
#         gridcolor = '#fafafa'
#     ),
#
#     font=dict(size=15),
#
#     # chart styling
#     plot_bgcolor = 'white',
#     paper_bgcolor = 'white',
#     showlegend=False,
#
#     # title info
#     title = dict(
#         text = 'NBA FT rate in the first 30 games of the 2020-21 and 2021-22 seasons',
#         font=dict(size=28),
#         x = 0.2,
#         y = 0.96
#     )
# )
#
# fig.show()

In [97]:
def make_scatter(opacity_1, opacity_2, metric_1, metric_2):
    # Draw one example figure indexing opacities
    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x = TOP15_DF[metric_1],
            y = TOP15_DF['PLAYER_NAME'],
            mode='markers',
            marker=dict(
                opacity=opacity_1,
                size=18,
                color='#d4d4d4'
            )
        )
    )

    fig.add_trace(
        go.Scatter(
            x = TOP15_DF[metric_2],
            y = TOP15_DF['PLAYER_NAME'],
            mode='markers',
            marker = dict(
                opacity = opacity_2,
                size=18
            )
        )
    )

    fig.update_layout(
        # dimensions
        height = 800,
        width = 1000,
        margin = dict(
            t = 50,
            b = 50,
            r = 10,
            l = 50,
            pad = 0
        ),

        # axes info
        xaxis = dict(
            domain = [0.2, 0.8],
            showgrid = False,
        ),
        yaxis = dict(
            categoryorder = 'total ascending',
            anchor = 'free',
            position = 0.05,
            side = 'right',
            gridcolor = '#fafafa'
        ),

        font=dict(size=15),

        # chart styling
        plot_bgcolor = 'white',
        paper_bgcolor = 'white',
        showlegend=False,

        # title info
        title = dict(
            text = 'NBA FT rate in the first 30 games of the 2020-21 and 2021-22 seasons',
            font=dict(size=28),
            x = 0.2,
            y = 0.96
        )
    )

    return fig

In [98]:
test_figure = make_scatter(0.5, 1., 'PAST_FTR', 'CURR_FTR')
test_figure.show()

In [ ]:
# create transition fucntion
# FTR diff by N_TRANSITION_FRAMES
# for each frame, move the second plotted graph over by that much

In [99]:
right_idx = 0
left_idx = 0

for n in range(N_FRAMES):
    # draw each figure
    right_idx -= 1
    left_idx = n

    figure = make_scatter(OPACITIES[left_idx], OPACITIES[right_idx],
                          'PAST_FTR', 'CURR_FTR')

    figure.write_image(f"plots/ftr_plot_img{n+1}.png")

In [101]:
[1,1].add([1,1])

AttributeError: 'list' object has no attribute 'add'